# ============================================================
# Models comparsion
# ============================================================

In [7]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import BatchNormalization, Dropout
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
from torch.utils.data import ConcatDataset, DataLoader
import torch.optim as optim
import pandas as pd
import time
import json




2025-11-03 01:29:36.282826: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# ============================================================
# Custom Neural Network
# ============================================================

In [30]:
best_lr = 0.01
best_batch_size = 32
best_arch = [512, 256, 128, 64]



# ======  Best Model Architecture ======
class FeedforwardNN(nn.Module):
    def __init__(self, layer_sizes, activation='relu'):
    
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
        
        if activation == 'relu':
            self.activation = F.relu
        elif activation == 'tanh':
            self.activation = torch.tanh
        elif activation == 'leaky_relu':
            self.activation = F.leaky_relu
        else:
            raise ValueError("Unsupported activation")

    def forward(self, x):
        x = x.view(x.size(0), -1)
        for layer in self.layers[:-1]:
            x = self.activation(layer(x))
        x = self.layers[-1](x)  
        return x
    
model = FeedforwardNN([784] + best_arch + [10], activation='relu')

# ====== 2. Prepare Combined Train + Validation Data ======
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
full_train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)

train_size = int(0.83 * len(full_train_data))   # ~50,000
val_size = len(full_train_data) - train_size    # ~10,000
train_data, val_data = random_split(full_train_data, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=64, shuffle=False)

test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
combined_dataset = ConcatDataset([train_data, val_data])
combined_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)

# ====== 3. Train Final Model ======
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

start_time = time.time()    #start time 

epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in combined_loader:
        images = images.view(images.size(0), -1)  # flatten 28x28 -> 784

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {total_loss/len(combined_loader):.4f} | "
    f"Acc: {100*correct/total:.2f}%")
    
    
end_time = time.time()   # end time
training_time_nn = end_time - start_time
print(f"\n🕒 Total Training Time: {training_time_nn:.2f} seconds")
# ====== 5. Evaluate on Test Set ======
model.eval()
y_true, y_pred = [], []
test_loss_nn = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(images.size(0), -1)
        outputs = model(images)
        loss = criterion(outputs, labels)  
        test_loss_nn += loss.item()
        preds = torch.argmax(outputs, dim=1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.numpy())

y_true, y_pred = np.array(y_true), np.array(y_pred)
loss__nn=nn.CrossEntropyLoss()(outputs, labels).item();
test_acc = np.mean(y_true == y_pred)
print(f"\n🎯 Final Test Accuracy: {test_acc*100:.2f}%")

Epoch [1/10] | Loss: 0.4667 | Acc: 84.90%
Epoch [2/10] | Loss: 0.1599 | Acc: 95.13%
Epoch [3/10] | Loss: 0.1153 | Acc: 96.48%
Epoch [4/10] | Loss: 0.0967 | Acc: 97.05%
Epoch [5/10] | Loss: 0.0809 | Acc: 97.44%
Epoch [6/10] | Loss: 0.0657 | Acc: 97.93%
Epoch [7/10] | Loss: 0.0556 | Acc: 98.27%
Epoch [8/10] | Loss: 0.0498 | Acc: 98.38%
Epoch [9/10] | Loss: 0.0456 | Acc: 98.59%
Epoch [10/10] | Loss: 0.0367 | Acc: 98.81%

🕒 Total Training Time: 93.27 seconds

🎯 Final Test Accuracy: 97.19%


In [31]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

train_images = np.expand_dims(train_images, axis=-1) # add channel dimension
test_images = np.expand_dims(test_images, axis=-1)

best_model = Sequential()
best_model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='leaky_relu', input_shape=(28, 28, 1)))
best_model.add(BatchNormalization())
best_model.add(Dropout(0.5))
best_model.add(Conv2D(filters=8, kernel_size=(3, 3), activation='leaky_relu'))
best_model.add(BatchNormalization())
best_model.add(Dropout(0.5))
best_model.add(Flatten()) # flatten the 2D arrays to 1D
best_model.add(Dense(10, activation='softmax'))
best_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
callbacks = [
    ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss'),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]
start_time = time.time()    #start time 
history = best_model.fit(train_images, train_labels, epochs=10, batch_size=8, validation_split=0.2, callbacks=callbacks)
end_time = time.time()   # end time
training_time_cnn = end_time - start_time

start_test_time = time.time()
cnn_test_accuracy = best_model.evaluate(test_images, test_labels, verbose=0)[1]
cnn_loss = best_model.evaluate(test_images, test_labels, verbose=0)[0]
end_test_time = time.time()
test_time = end_test_time - start_test_time

print(f'Best CNN Test Accuracy: {cnn_test_accuracy:.4f}')
print(f'Best CNN Test Loss: {cnn_loss:.4f}')
print(f'Best CNN Test Time: {test_time:.2f} seconds')
print(f"\n🕒 Total Training Time: {training_time_cnn:.2f} seconds")

/home/ziademad/Assigment_2/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-11-02 20:21:07.595352: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 150528000 exceeds 10% of free system memory.


Epoch 1/10
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 28s 5ms/step - accuracy: 0.9194 - loss: 0.2889 - val_accuracy: 0.9716 - val_loss: 0.0989
Epoch 2/10
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 0.9606 - loss: 0.1282 - val_accuracy: 0.9663 - val_loss: 0.1051
Epoch 3/10
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.9674 - loss: 0.1050 - val_accuracy: 0.9810 - val_loss: 0.0673
Epoch 4/10
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 0.9708 - loss: 0.0945 - val_accuracy: 0.9822 - val_loss: 0.0633
Epoch 5/10
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 0.9735 - loss: 0.0863 - val_accuracy: 0.9787 - val_loss: 0.0728
Epoch 6/10
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 0.9736 - loss: 0.0842 - val_accuracy: 0.9843 - val_loss: 0.0574
Epoch 7/10
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 0.9748 - loss: 0.0799 - val_accuracy: 0.9833 - val_loss: 0.0590
Epoch 8/10
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.9754 - loss: 0

# ============================================================
# model comparison table
# ============================================================

In [8]:
with open("logreg_results.json", "r") as f:
    lr_results = json.load(f)

with open("softmax_results.json", "r") as f:
    softmax_results = json.load(f)


df_results = pd.DataFrame([
    {'Model': 'Logistic Regression (Manual)', 'Test Accuracy (%)': lr_results["test_accuracy"] * 100, 'Test Loss': lr_results["test_loss"], 'Training Time (s)': lr_results["training_time_lr"]},
    {'Model': 'Softmax Regression (Manual)', 'Test Accuracy (%)': softmax_results["test_accuracy_manual"] * 100, 'Test Loss': softmax_results["test_loss_manual"], 'Training Time (s)': softmax_results["train_time_manual"]},
    {'Model': 'Softmax Regression (Built-in)', 'Test Accuracy (%)': softmax_results["test_accuracy_builtin"] * 100, 'Test Loss': softmax_results["test_loss_builtin"], 'Training Time (s)': softmax_results["training_time_builtin"]},
    {'Model': 'Neural Network (Best Params)', 'Test Accuracy (%)': test_acc * 100, 'Test Loss': loss__nn, 'Training Time (s)': training_time_nn},
    {'Model': 'CNN (Best Model)', 'Test Accuracy (%)': cnn_test_accuracy * 100, 'Test Loss': cnn_loss, 'Training Time (s)': training_time_cnn}
])

display(df_results)

NameError: name 'test_acc' is not defined

### 🚀 Model Comparison Summary (Accuracy vs Training Time)

| Model | Accuracy | Training Time |
|------|---------|---------------|
| Logistic Regression (Manual) | **99.76%** | **0.92s** |
| Softmax Regression (Built-in) | 90.76% | **0.0008s** |
| Neural Network (Best Params) | 97.19% | 93.26s |
| CNN (Best Model) | 98.28% | 266.46s |

#### 📌 Key Takeaways
- **Logistic Regression** achieved the highest accuracy with extremely fast training.
- **Built-in Softmax** is ideal for instant baseline performance checks.
- **Neural Network** improves learning for non-linear patterns, but training is slow.
- **CNN** excels in spatial feature extraction (best for images) but has the highest compute cost.

#### 🧠 Usage Guide
| When | Best Model |
|------|-----------|
Simple / linearly separable data | Logistic Regression |
Fast prototyping | Built-in Softmax |
Non-linear complex patterns | Neural Network |
Image / spatial structure | CNN |

> ✅ Simple models = fast + accurate for simple patterns  
> ✅ Deep models = needed only when spatial or deeper feature learning matters  
